In [47]:
import pandas as pd
import re
import numpy as np
import string

In [48]:
remedy_file0 = pd.read_csv(r"C:\Users\A174411\Onedrive - Progressive Corporation\groupProject\Custom SQL Query (InformationTechnology)_Custom SQL Query.csv", dtype='object')

In [49]:
remedy_file = remedy_file0.copy(deep=True)
print(remedy_file.shape)
print(remedy_file.columns)

(225491, 12)
Index(['Assignee', 'Priority', 'Record Type', 'RecordID', 'Reported_Time',
       'Resolved_Date', 'Status', 'Status Reason', 'Submission Type',
       'Support_Group', 'Support_Org', 'Text'],
      dtype='object')


In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [52]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [53]:
# REPLACES NULL WITH NaN
remedy_file['Text'].replace('', np.nan, inplace=True)
#remedy_file.head()

# DROPS NaN
remedy_file.dropna(subset=['Text'], inplace=True)
remedy_file.head()

,Assignee,Priority,Record Type,RecordID,Reported_Time,Resolved_Date,Status,Status Reason,Submission Type,Support_Group,Support_Org,Text
6,William G Pritchard,High,Incident,INC000012547401,5/14/2022 3:35,5/14/2022 7:15,Closed,Automated Resolution Reported,Automated,Systems & Infrastructure T2,Computing,<SERVER> Reported no UF data for past hour
7,William G Pritchard,High,Incident,INC000012547622,5/14/2022 5:36,5/14/2022 6:11,Closed,Automated Resolution Reported,Automated,Systems & Infrastructure T2,Computing,<SERVER> Reported no UF data for past hour
8,William G Pritchard,High,Incident,INC000012493989,4/27/2022 6:25,4/27/2022 8:59,Closed,Automated Resolution Reported,Automated,Systems & Infrastructure T2,Computing,No KBA found for this. Received help from Ben ...
9,William G Pritchard,High,Incident,INC000012493991,4/27/2022 6:25,4/27/2022 8:59,Closed,Automated Resolution Reported,Automated,Systems & Infrastructure T2,Computing,No KBA found for this. Received help from Ben ...
10,William G Pritchard,High,Incident,INC000012480898,4/23/2022 5:39,4/23/2022 5:51,Closed,Automated Resolution Reported,Automated,Systems & Infrastructure T2,Computing,"No KBA in existence for this issue, but INC000..."


In [54]:
# CLEANS THE TEXT
# clean_remedy_file = remedy_file.translate(str.maketrans('', '', string.punctuation)) #Error. 'DataFrame' object has no attribute 'translate'

remedy_file["clean_remedy_file"] = remedy_file['Text'].str.replace('[^\w\s]',' ') #works!
remedy_file["clean_remedy_file"]

c:\miniconda37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


6                 SERVER  Reported no UF data for past hour
7                 SERVER  Reported no UF data for past hour
8         No KBA found for this  Received help from Ben ...
9         No KBA found for this  Received help from Ben ...
10        No KBA in existence for this issue  but INC000...
                                ...                        
225476    HOBICDBD HOBICRDV OBIDBD03 ARCHIVELOG 20210428...
225477                     Health Check on S65AA338 Server 
225478                    Please reboot DMZ server SMVQWP2D
225479    Verifying cabling reseat if necessary for devi...
225483    Per Cloud Ops  David Garcia  Event Monitoring ...
Name: clean_remedy_file, Length: 155393, dtype: object

In [57]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [58]:
#remedy_file["clean_remedy_file"] = remedy_file["clean_remedy_file"].apply(lambda row: nltk.word_tokenize(row['Text']))

stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(remedy_file["clean_remedy_file"])
filtered_Text = [word for word in word_tokens if not word.lower() in stop_words]
filtered_Text = []

for word in word_tokens:
    if word not in stop_words:
        filtered_Text.append(word)
  
word_tokens
filtered_Text

TypeError: expected string or bytes-like object

In [7]:
import matplotlib.pyplot as plt
from nltk.probability import FreqDist